In [ ]:
from flask import Flask, request, redirect, jsonify
import oracledb

app = Flask(__name__)

conexion = oracledb.connect(
    user="banquetes_admin",
    password="banquetes2025",
    dsn="localhost/XEPDB1"
)
cursor = conexion.cursor()

@app.route("/registrarUsuario", methods=["POST"])
def registrar_usuario():
    data = request.get_json()

    usuario = data["usuario"]
    password = data["password"]
    email = data["email"]
    tipo_usuario = data["tipo_usuario"]
    nombre = data["nombre"]
    apellido = data["apellido"]
    telefono = data.get("telefono")
    rfc = data.get("rfc")
    direccion = data.get("direccion")

    try:
        # Insertar en USUARIOS
        cursor.execute("""
            INSERT INTO usuarios (usuario, password, email, tipo_usuario)
            VALUES (:1, :2, :3, :4)
            RETURNING id_usuario INTO :5
        """, (usuario, password, email, tipo_usuario, cursor.var(int)))
        id_usuario = cursor.getimplicitresults()[0][0]

        # Si es cliente, insertar en CLIENTES
        if tipo_usuario == "CLIENTE":
            cursor.execute("""
                INSERT INTO clientes (id_usuario, nombre, apellido, telefono, rfc, direccion)
                VALUES (:1, :2, :3, :4, :5, :6)
            """, (id_usuario, nombre, apellido, telefono, rfc, direccion))

        conexion.commit()
        return jsonify({"mensaje": "Usuario registrado con éxito", "id_usuario": id_usuario}), 201
    except oracledb.DatabaseError as e:
        conexion.rollback()
        return jsonify({"error": str(e)}), 500

@app.route("/usuarios", methods=["GET"])
def obtenerUsuarios():
    cursor.execute("""
        SELECT * FROM usuarios
    """)

    columnas = [col[0].lower() for col in cursor.description]
    resultados = [dict(zip(columnas, fila)) for fila in cursor.fetchall()]
    return jsonify(resultados)


if __name__ == '__main__':
    app.run()
